In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
import pyspark
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import countDistinct, col

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# User Latent Features

In [4]:
user_factors = pd.read_csv('data/user_factors.csv')

In [5]:
user_factors.head()

,id,features
0,640,"[0.5853673219680786, 1.1955639123916626, 0.495..."
1,650,"[0.7357923984527588, 1.1214193105697632, 0.211..."
2,660,"[0.41824525594711304, 0.8636453151702881, 0.40..."
3,670,"[0.0, 0.23701752722263336, 0.10014515370130539..."
4,680,"[0.2757687568664551, 0.9952836632728577, 0.413..."


# Movie Latent Features

In [6]:
movie_factors = pd.read_csv('data/movie_factors.csv')

In [7]:
movie_factors['features'][0]

'[0.1470944583415985, 1.0527691841125488, 0.06695104390382767, 0.2533167600631714, 0.17517654597759247, 0.5150220990180969, 0.18830619752407074, 0.5612077713012695, 0.11690513044595718, 0.7164477705955505]'

# Requests Data

In [ ]:
requests_df = pd.read_csv('data/requests.csv')

In [ ]:
requests_df.head()

# Training Data

In [ ]:
training = pd.read_csv('data/training.csv')

In [ ]:
training.head()

In [ ]:
training['movie'].nunique()

In [ ]:
print('Training data statistics: ')
training.describe()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

ax.hist(training.groupby('user').count()['rating'], bins=150)
ax.set_xlabel('Num of ratings given')
ax.set_ylabel('count of user');

# Movie Descriptions

In [22]:
movies = pd.read_csv('data/movies.dat', sep='::', engine='python', names=['movie_id', 'title', 'genre'])

In [23]:
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [24]:
movies['genre'] = movies['genre'].apply(lambda x: x.replace('|', ' '))

In [25]:
movies.describe()

,movie_id
count,3883.000000
mean,1986.049446
std,1146.778349
min,1.000000
25%,982.500000
50%,2010.000000
75%,2980.500000
max,3952.000000


In [26]:
movies['genre'].unique()

array(["Animation Children's Comedy", "Adventure Children's Fantasy",
       'Comedy Romance', 'Comedy Drama', 'Comedy',
       'Action Crime Thriller', "Adventure Children's", 'Action',
       'Action Adventure Thriller', 'Comedy Drama Romance',
       'Comedy Horror', "Animation Children's", 'Drama',
       'Action Adventure Romance', 'Drama Thriller', 'Drama Romance',
       'Thriller', 'Action Comedy Drama', 'Crime Drama Thriller',
       'Drama Sci-Fi', 'Romance', 'Adventure Sci-Fi', 'Adventure Romance',
       "Children's Comedy Drama", 'Documentary', 'Drama War',
       'Action Crime Drama', 'Action Adventure', 'Crime Thriller',
       "Animation Children's Musical Romance", 'Action Drama Thriller',
       "Children's Comedy", 'Drama Mystery', 'Sci-Fi Thriller',
       'Action Comedy Crime Horror Thriller', 'Drama Musical',
       'Crime Drama Romance', 'Adventure Drama', 'Action Thriller',
       "Adventure Children's Comedy Musical", 'Action Drama War',
       'Action Adventur

In [27]:
movies.set_index('movie_id', inplace=True)

In [28]:
movies_df = movies.copy().drop('title', axis=1)

In [29]:
movies_df.to_csv('data/movies.csv')

In [30]:
tf = TfidfVectorizer()
movies_tfidf = tf.fit_transform(movies_df['genre'])
movies_indices = pd.Series(movies_df.index)

In [31]:
# movies_sim_mat = pd.DataFrame(cosine_similarity(movies_tfidf), index=movies_indices, columns=movies_indices)

In [153]:
movies_sim_mat = pd.read_csv('data/movies_sim_mat.csv', index_col='movie_id')

In [150]:
movies_sim_mat.head(1)

,movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,3903,3904,3905,3906,3907,3908,3909,3910,3911,3912,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,1,1.0,0.305525,0.197372,0.260194,0.344351,0.0,0.197372,0.425153,0.0,0.0,0.176755,0.18456,0.938841,0.0,0.0,0.0,0.0,0.0,0.344351,0.0,0.178778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.627557,0.0,0.0,0.0,0.344351,0.197372,0.0,0.0,0.0,0.0,0.0,0.260194,0.0,0.0,0.689027,0.0,...,0.0,0.0,0.344351,0.0,0.0,0.0,0.197372,0.0,0.344351,0.260194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.344351,0.344351,0.0,0.344351,0.344351,0.0,0.0,0.18456,0.344351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.344351,0.260194,0.80146,0.0,0.0,0.344351,0.0,0.0,0.0,0.0


In [85]:
# movies_sim_mat.to_csv('data/movies_sim_mat.csv')
print('Movies Cosine Similarity Matrix: ')
movies_sim_mat.head(10)

Movies Cosine Similarity Matrix: 


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,3903,3904,3905,3906,3907,3908,3909,3910,3911,3912,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
movie_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.305525,0.197372,0.260194,0.344351,0.000000,0.197372,0.425153,0.000000,0.000000,0.176755,0.184560,0.938841,0.000000,0.000000,0.000000,0.000000,0.000000,0.344351,0.000000,0.178778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.627557,0.000000,0.000000,0.0,0.344351,0.197372,0.000000,0.000000,0.000000,0.000000,0.000000,0.260194,0.000000,0.000000,0.689027,0.000000,0.000000,...,0.000000,0.000000,0.344351,0.000000,0.000000,0.0,0.197372,0.000000,0.344351,0.260194,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.344351,0.344351,0.0,0.344351,0.344351,0.000000,0.000000,0.184560,0.344351,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.344351,0.260194,0.801460,0.000000,0.000000,0.344351,0.000000,0.000000,0.000000,0.000000
2,0.305525,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.718623,0.000000,0.320890,0.000000,0.000000,0.325428,0.000000,0.319540,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.287244,0.000000,0.000000,0.000000,0.379045,0.409075,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.382454,0.000000,0.000000,0.000000,0.238835,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.651211,0.000000,0.000000,0.0,0.000000,0.000000,0.287244,0.287244,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.538118,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.197372,0.000000,1.000000,0.433093,0.573172,0.000000,1.000000,0.000000,0.000000,0.000000,0.895543,0.307200,0.000000,0.000000,0.450274,0.000000,0.700684,0.000000,0.573172,0.000000,0.297576,0.000000,0.000000,0.000000,0.700684,0.000000,0.000000,0.819435,0.000000,0.000000,0.000000,0.000000,0.534124,0.264266,0.700684,0.000000,0.0,0.573172,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.433093,0.700684,0.000000,0.315353,0.700684,0.000000,...,0.000000,0.000000,0.573172,0.000000,0.000000,0.0,1.000000,0.000000,0.573172,0.433093,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.573172,0.573172,0.0,0.573172,0.573172,0.000000,0.000000,0.307200,0.573172,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.573172,0.433093,0.000000,0.000000,0.000000,0.573172,0.000000,0.000000,0.000000,0.000000
4,0.260194,0.000000,0.433093,1.000000,0.755606,0.000000,0.433093,0.000000,0.000000,0.000000,0.679323,0.404979,0.000000,0.655026,0.000000,0.343133,0.339626,0.000000,0.755606,0.000000,0.687095,0.232383,0.000000,0.225132,0.339626,0.655026,0.655026,0.000000,0.000000,0.655026,0.655026,0.225132,0.000000,0.610184,0.339626,0.655026,0.0,0.755606,0.433093,0.655026,0.263178,0.232940,0.655026,0.000000,1.000000,0.339626,0.000000,0.000000,0.339626,0.000000,...,0.655026,0.655026,0.755606,0.000000,0.655026,0.0,0.433093,0.252066,0.755606,1.000000,0.0,0.655026,0.655026,0.655026,0.0,0.0,0.0,0.229773,0.755606,0.755606,0.0,0.755606,0.755606,0.000000,0.000000,0.404979,0.755606,0.0,0.0,0.0,0.0,0.0,0.0,0.343133,0.000000,0.0,0.0,0.0,0.0,0.0,0.755606,1.000000,0.000000,0.262005,0.000000,0.755606,0.655026,0.655026,0.655026,0.343133
5,0.344351,0.000000,0.573172,0.755606,1.000000,0.000000,0.573172,0.000000,0.000000,0.000000,0.513301,0.535965,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.519173,0.000000,0.000000,0.000000,0.000000,0.0

In [86]:
movie_id = 2

In [95]:
# find similarity of GIVEN movie_id and movie_id 193
# USE .LOC BECAUSE INDEXING THE NAMES
movies_sim_mat.loc[movie_id]

1       0.305525
2       1.000000
3       0.000000
4       0.000000
5       0.000000
6       0.000000
7       0.000000
8       0.718623
9       0.000000
10      0.320890
11      0.000000
12      0.000000
13      0.325428
14      0.000000
15      0.319540
16      0.000000
17      0.000000
18      0.000000
19      0.000000
20      0.000000
21      0.000000
22      0.000000
23      0.000000
24      0.000000
25      0.000000
26      0.000000
27      0.000000
28      0.000000
29      0.287244
30      0.000000
          ...   
3923    0.000000
3924    0.000000
3925    0.000000
3926    0.287244
3927    0.287244
3928    0.000000
3929    0.000000
3930    0.000000
3931    0.000000
3932    0.000000
3933    0.000000
3934    0.000000
3935    0.000000
3936    0.000000
3937    0.000000
3938    0.000000
3939    0.000000
3940    0.000000
3941    0.000000
3942    0.000000
3943    0.000000
3944    0.000000
3945    0.538118
3946    0.000000
3947    0.000000
3948    0.000000
3949    0.000000
3950    0.0000

In [35]:
np.sort(movies_sim_mat[movie_id])

array([0., 0., 0., ..., 1., 1., 1.])

In [36]:
movies_sim_mat.loc[2, 1897]

0.0

In [168]:
movies_sim_mat.loc[movies_sim_mat[f'{movie_id}'] >= 1, f'{movie_id}'][1:].index.values

array([  56,   60,  126, 1009, 1967, 2005, 2043, 2161, 2162, 2399])

In [121]:
movies_sim_mat[(movies_sim_mat.loc[movie_id].sort_values(ascending=False)>=1).values == True]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,3903,3904,3905,3906,3907,3908,3909,3910,3911,3912,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
movie_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.305525,0.197372,0.260194,0.344351,0.000000,0.197372,0.425153,0.000000,0.000000,0.176755,0.184560,0.938841,0.000000,0.000000,0.000000,0.000000,0.000000,0.344351,0.000000,0.178778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.627557,0.000000,0.000000,0.0,0.344351,0.197372,0.000000,0.000000,0.000000,0.000000,0.000000,0.260194,0.000000,0.000000,0.689027,0.000000,0.000000,...,0.000000,0.000000,0.344351,0.000000,0.000000,0.0,0.197372,0.000000,0.344351,0.260194,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.344351,0.344351,0.0,0.344351,0.344351,0.000000,0.000000,0.184560,0.344351,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.344351,0.260194,0.801460,0.000000,0.000000,0.344351,0.000000,0.000000,0.000000,0.000000
2,0.305525,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.718623,0.000000,0.320890,0.000000,0.000000,0.325428,0.000000,0.319540,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.287244,0.000000,0.000000,0.000000,0.379045,0.409075,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.382454,0.000000,0.000000,0.000000,0.238835,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.651211,0.000000,0.000000,0.0,0.000000,0.000000,0.287244,0.287244,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.538118,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.197372,0.000000,1.000000,0.433093,0.573172,0.000000,1.000000,0.000000,0.000000,0.000000,0.895543,0.307200,0.000000,0.000000,0.450274,0.000000,0.700684,0.000000,0.573172,0.000000,0.297576,0.000000,0.000000,0.000000,0.700684,0.000000,0.000000,0.819435,0.000000,0.000000,0.000000,0.000000,0.534124,0.264266,0.700684,0.000000,0.0,0.573172,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.433093,0.700684,0.000000,0.315353,0.700684,0.000000,...,0.000000,0.000000,0.573172,0.000000,0.000000,0.0,1.000000,0.000000,0.573172,0.433093,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.573172,0.573172,0.0,0.573172,0.573172,0.000000,0.000000,0.307200,0.573172,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.573172,0.433093,0.000000,0.000000,0.000000,0.573172,0.000000,0.000000,0.000000,0.000000
4,0.260194,0.000000,0.433093,1.000000,0.755606,0.000000,0.433093,0.000000,0.000000,0.000000,0.679323,0.404979,0.000000,0.655026,0.000000,0.343133,0.339626,0.000000,0.755606,0.000000,0.687095,0.232383,0.000000,0.225132,0.339626,0.655026,0.655026,0.000000,0.000000,0.655026,0.655026,0.225132,0.000000,0.610184,0.339626,0.655026,0.0,0.755606,0.433093,0.655026,0.263178,0.232940,0.655026,0.000000,1.000000,0.339626,0.000000,0.000000,0.339626,0.000000,...,0.655026,0.655026,0.755606,0.000000,0.655026,0.0,0.433093,0.252066,0.755606,1.000000,0.0,0.655026,0.655026,0.655026,0.0,0.0,0.0,0.229773,0.755606,0.755606,0.0,0.755606,0.755606,0.000000,0.000000,0.404979,0.755606,0.0,0.0,0.0,0.0,0.0,0.0,0.343133,0.000000,0.0,0.0,0.0,0.0,0.0,0.755606,1.000000,0.000000,0.262005,0.000000,0.755606,0.655026,0.655026,0.655026,0.343133
5,0.344351,0.000000,0.573172,0.755606,1.000000,0.000000,0.573172,0.000000,0.000000,0.000000,0.513301,0.535965,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.519173,0.000000,0.000000,0.000000,0.000000,0.0

In [37]:
# sorts movie_ids by similarity values from greatest to smallest
# try:

movies_sim_mat.loc[movies_sim_mat.loc[movie_id].sort_values(ascending=False)>=1]
# except:
#     movies_sim_mat.loc[movies_sim_mat[movie_id].sort_values(ascending=False) >= 0.9]

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,3903,3904,3905,3906,3907,3908,3909,3910,3911,3912,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
movie_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,0.305525,1.0,0.0,0.0,0.0,0.0,0.0,0.718623,0.0,0.32089,0.0,0.0,0.325428,0.0,0.31954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.287244,0.0,0.0,0.0,0.379045,0.409075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.382454,0.0,0.0,0.0,0.238835,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651211,0.0,0.0,0.0,0.0,0.0,0.287244,0.287244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538118,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56,0.305525,1.0,0.0,0.0,0.0,0.0,0.0,0.718623,0.0,0.32089,0.0,0.0,0.325428,0.0,0.31954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.287244,0.0,0.0,0.0,0.379045,0.409075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.382454,0.0,0.0,0.0,0.238835,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651211,0.0,0.0,0.0,0.0,0.0,0.287244,0.287244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538118,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,0.305525,1.0,0.0,0.0,0.0,0.0,0.0,0.718623,0.0,0.32089,0.0,0.0,0.325428,0.0,0.31954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.287244,0.0,0.0,0.0,0.379045,0.409075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.382454,0.0,0.0,0.0,0.238835,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651211,0.0,0.0,0.0,0.0,0.0,0.287244,0.287244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538118,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,0.305525,1.0,0.0,0.0,0.0,0.0,0.0,0.718623,0.0,0.32089,0.0,0.0,0.325428,0.0,0.31954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.287244,0.0,0.0,0.0,0.379045,0.409075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.382454,0.0,0.0,0.0,0.238835,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651211,0.0,0.0,0.0,0.0,0.0,0.287244,0.287244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538118,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1009,0.305525,1.0,0.0,0.0,0.0,0.0,0.0,0.718623,0.0,0.32089,0.0,0.0,0.325428,0.0,0.31954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.287244,0.0,0.0,0.0,0.379045,0.409075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.382454,0.0,0.0,0.0,0.238835,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651211,0.0,0.0,0.0,0.0,0.0,0.287244,0.287244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538118,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1967,0.305525,1.0,0.0,0.0,0.0,0.0,0.0,0.718623,0.0,0.32089,0.0,0.0,0.325428,0.0,0.31954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.287244,0.0,0.0,0.0,0.379045,0.409075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.382454,0.0,0.0,0.0,0.238835,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651211,0.0,0.0,0.0,0.0,0.0,0.287244,0.287244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538118,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2005,0.305525,1.0,0.0,0.0,0.0,0.0,0.0,0.718623,0.0,0.32089,0.0,0.0,0.325428,0.0,0.31954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.287244,0.0,0.0,0.0,0.379045,0.409075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.382454,0.0,0.0,0.0,0.238835,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651211,0.0,0.0,0.0,0.0,0.0,0.287244,0.287244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538118,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2043,0.305525,1.0,0.0,0.0,0.0,0.0,0.0,0.718623,0.0,0.32089,0.0,0.0,0.325428,0.0,0.31954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.287244,0.0,0.

In [38]:
movies_sim_mat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3883 entries, 1 to 3952
Columns: 3883 entries, 1 to 3952
dtypes: float64(3883)
memory usage: 115.2 MB


In [39]:
movies_sim_mat.loc[762, 761]

0.0

In [40]:
similar_movie_ids = movies_sim_mat.loc[round(movies_sim_mat[movie_id].sort_values(ascending=False), 5) >= 1][1:5].index.values

In [75]:
movies_sim_mat.loc[movies_sim_mat[movie_id].sort_values(ascending=False) >= 1][1:5]

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,3903,3904,3905,3906,3907,3908,3909,3910,3911,3912,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
movie_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
56,0.305525,1.0,0.0,0.0,0.0,0.0,0.0,0.718623,0.0,0.32089,0.0,0.0,0.325428,0.0,0.31954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.287244,0.0,0.0,0.0,0.379045,0.409075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.382454,0.0,0.0,0.0,0.238835,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651211,0.0,0.0,0.0,0.0,0.0,0.287244,0.287244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538118,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,0.305525,1.0,0.0,0.0,0.0,0.0,0.0,0.718623,0.0,0.32089,0.0,0.0,0.325428,0.0,0.31954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.287244,0.0,0.0,0.0,0.379045,0.409075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.382454,0.0,0.0,0.0,0.238835,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651211,0.0,0.0,0.0,0.0,0.0,0.287244,0.287244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538118,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,0.305525,1.0,0.0,0.0,0.0,0.0,0.0,0.718623,0.0,0.32089,0.0,0.0,0.325428,0.0,0.31954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.287244,0.0,0.0,0.0,0.379045,0.409075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.382454,0.0,0.0,0.0,0.238835,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651211,0.0,0.0,0.0,0.0,0.0,0.287244,0.287244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538118,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1009,0.305525,1.0,0.0,0.0,0.0,0.0,0.0,0.718623,0.0,0.32089,0.0,0.0,0.325428,0.0,0.31954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.287244,0.0,0.0,0.0,0.379045,0.409075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.382454,0.0,0.0,0.0,0.238835,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651211,0.0,0.0,0.0,0.0,0.0,0.287244,0.287244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538118,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
movie_factors_df = pd.read_csv('data/movie_factors_all.csv', index_col='id')

In [43]:
from tqdm import tqdm
from ast import literal_eval

In [44]:
items = []

for i in tqdm(similar_movie_ids):
    i_features = literal_eval(movie_factors_df.loc[i, 'features'])
    item = np.array(i_features)
    items.append(item)

100%|██████████| 4/4 [00:00<00:00, 897.47it/s]


In [45]:
np.mean(items, axis=0)

array([0.10310895, 0.38205667, 0.21001234, 0.25752176, 0.72520935,
       0.18925987, 0.58186877, 0.2852918 , 0.33939688, 0.20164722])

In [ ]:
len(items)

# User Demographics

In [9]:
# users_demo = pd.read_csv('data/users.dat', sep='::', engine='python', names=['user_id', 'gender', 'age', 'occupation', 'zipcode'])

# users_demo.head()

# users_demo.describe()

# users_demo.info()

# users_demo.set_index('user_id', inplace=True)

# users_demo.head(10)

# users_indices = pd.Series(users_demo.index)

# users_demo['gender'] = users_demo['gender'].map({'M':1, 'F':0})

# import re

# users_demo['zipcode'] = users_demo['zipcode'].str.replace('-[0-9][0-9][0-9][0-9]', '').astype(int)

# users_demo.info()

# users_demo.drop('zipcode', axis=1, inplace=True)

# users_demo.info()

# users_demo.to_csv('data/users.csv')

In [9]:
# users_sim_mat = pd.DataFrame(cosine_similarity(testing_users), index=users_indices, columns=users_indices)

# users_sim_mat.to_csv('data/users_sim_mat.csv')

In [10]:
users_sim_mat = pd.read_csv('data/users_sim_mat.csv', index_col='user_id')

In [11]:
user_factors_df = pd.read_csv('data/user_factors_all.csv', index_col='id')

In [12]:
user_factors_df = user_factors_df.drop('Unnamed: 0', axis=1)

In [46]:
user_factors_df

,features
id,
640,"[1.3606634140014648, 0.2861190736293793, 0.865..."
650,"[0.9298866391181946, 0.5678400993347168, 0.175..."
660,"[0.6424887776374817, 0.7468441724777222, 0.516..."
670,"[1.1447361707687378, 0.8470364212989807, 0.586..."
680,"[0.8505991101264954, 1.0462939739227295, 0.013..."
690,"[1.0453484058380127, 0.6749264597892761, 0.611..."
700,"[0.5903452038764954, 0.8465031385421753, 1.375..."
710,"[0.8934944272041321, 0.7182586193084717, 0.703..."
720,"[0.6489324569702148, 0.6614680886268616, 0.0, ..."


In [13]:
print('User Cosine Similarity Matrix: ')
users_sim_mat.head(10)

User Cosine Similarity Matrix: 


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,5991,5992,5993,5994,5995,5996,5997,5998,5999,6000,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.368979,0.596915,0.251205,0.698953,0.274204,0.127829,0.519950,0.641451,0.127881,0.139194,0.519950,0.121556,0.099463,0.363840,0.099504,0.119357,0.261733,0.995086,0.572647,0.734804,0.712799,0.099463,0.364110,0.312529,0.363840,0.491488,0.139194,0.292599,0.292714,0.222120,0.099504,0.165432,0.099504,0.121556,0.217178,0.430660,0.312988,0.312529,0.099479,0.312988,0.397744,0.519950,0.444634,0.427101,0.790206,0.312529,0.255261,0.634062,0.178536,...,0.580070,0.312988,0.461934,0.127829,0.127881,0.099504,0.364110,0.312529,0.139194,0.444634,0.364110,0.099484,0.444730,0.597266,0.292714,0.099504,0.524067,0.312529,0.520288,0.099463,0.483256,0.483256,0.699294,0.121556,0.430660,0.121556,0.292714,0.127829,0.099424,0.397961,0.519950,0.641451,0.099424,0.519950,0.139194,0.266093,0.312529,0.312529,0.139194,0.641451,0.099504,0.251205,0.346622,0.572647,0.139194,0.597266,0.121586,0.117254,0.099504,0.328722
2,0.368979,1.000000,0.965721,0.992316,0.922391,0.994837,0.968930,0.985550,0.949497,0.968835,0.971593,0.985550,0.967390,0.961480,0.999759,0.961382,0.966833,0.993459,0.368865,0.973028,0.900970,0.914301,0.961480,0.999839,0.997541,0.999759,0.990557,0.971593,0.996672,0.996582,0.988177,0.961382,0.977665,0.961382,0.967390,0.987165,0.997592,0.998076,0.997541,0.961526,0.998076,0.999289,0.985550,0.996560,0.997849,0.860619,0.997541,0.992609,0.952061,0.980225,...,0.970993,0.998076,0.994635,0.968930,0.968835,0.961382,0.999839,0.997541,0.971593,0.996560,0.999839,0.961533,0.996419,0.965700,0.996582,0.961382,0.984896,0.997541,0.985571,0.961480,0.991961,0.991961,0.922305,0.967390,0.997592,0.967390,0.996582,0.968930,0.961300,0.999505,0.985550,0.949497,0.961300,0.985550,0.971593,0.994060,0.997541,0.997541,0.971593,0.949497,0.961382,0.992316,0.999711,0.973028,0.971593,0.965700,0.967247,0.966133,0.961382,0.998851
3,0.596915,0.965721,1.000000,0.926369,0.990997,0.934525,0.871948,0.995692,0.998391,0.871325,0.876855,0.995692,0.868748,0.857618,0.964497,0.856989,0.867611,0.929862,0.597376,0.999552,0.982710,0.988099,0.857618,0.963891,0.948591,0.964497,0.992091,0.876855,0.941779,0.941188,0.914613,0.856989,0.889836,0.856989,0.868748,0.912790,0.980497,0.948126,0.948591,0.857539,0.948126,0.973530,0.995692,0.983906,0.979727,0.963277,0.948591,0.928094,0.998799,0.895264,...,0.999186,0.948126,0.986661,0.871948,0.871325,0.856989,0.963891,0.948591,0.876855,0.983906,0.963891,0.857503,0.983405,0.999412,0.941188,0.856989,0.996074,0.948591,0.995103,0.857618,0.990806,0.990806,0.990410,0.868748,0.980497,0.868748,0.941188,0.871948,0.857674,0.973407,0.995692,0.998391,0.857674,0.995692,0.876855,0.932143,0.948591,0.948591,0.876855,0.998391,0.856989,0.926369,0.959286,0.999552,0.876855,0.999412,0.868201,0.866033,0.856989,0.953936
4,0.251205,0.992316,0.926369,1.000000,0.867663,0.999476,0.992087,0.957262,0.903545,0.991864,0.993231,0.957262,0.991292,0.988102,0.992835,0.987878,0.990994,0.999700,0.252144,0.937209,0.840629,0.857443,0.988102,0.992725,0.997415,0.992835,0.966265,0.993231,0.999055,0.998832,0.999542,0.987878,0.996154,0.987878,0.991292,0.999235,0.981533,0.997690,0.997415,0.988122,0.997690,0.987834,0.957262,0.978685,0.982274,0.791227,0.997415,0.999837,0.907251,0.996987,...,0.933960,0.997690,0.974294,0.992087,0.991864,0.987878,0.992725,0.997415,0.993231,0.978685,0.992725,0.988120,0.978440,0.926161,0.998832,0.987878,0.955992,0.997415,0.957093,0.988102,0.968780,0.968780,0.867400,0.991292,0.981533,0.991292,0.998832,0.992087,0.987966,0.987953,0.957262,0.903545,0.987966,0.957262,0.993231,0.9998

In [ ]:
# users_sim_mat.describe()

In [56]:
user_id = 1

In [58]:
users_sim_mat[round(users_sim_mat.loc[user_id].sort_values(ascending=False), 5) >= 1][1:].index.values

/Users/cindywong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [59]:
user_factors_df.head()

,features
id,
640,"[1.3606634140014648, 0.2861190736293793, 0.865..."
650,"[0.9298866391181946, 0.5678400993347168, 0.175..."
660,"[0.6424887776374817, 0.7468441724777222, 0.516..."
670,"[1.1447361707687378, 0.8470364212989807, 0.586..."
680,"[0.8505991101264954, 1.0462939739227295, 0.013..."


In [60]:
len(user_factors_df.columns)

1

In [74]:
(users_sim_mat.loc[user_id].sort_values(ascending=False) >=1)

1        True
4575     True
1365     True
4883     True
1421     True
1434     True
1508     True
1568     True
4690     True
119      True
4606     True
4572     True
5063     True
4549     True
4493     True
1929     True
99       True
2133     True
86       True
2137     True
75       True
2155     True
1348     True
1195     True
5989     True
629      True
5756     True
5844     True
468      True
470      True
        ...  
2996    False
1221    False
5269    False
5730    False
5394    False
5652    False
715     False
189     False
3225    False
1582    False
4304    False
1653    False
1056    False
5074    False
3594    False
1458    False
3780    False
1296    False
2169    False
5390    False
4633    False
1573    False
5777    False
719     False
1509    False
1919    False
2120    False
1200    False
5228    False
4707    False
Name: 1, Length: 6040, dtype: bool

In [63]:
users_sim_mat.loc[round(users_sim_mat.loc[user_id].sort_values(ascending=False), 5) >= 1][1:11]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [21]:
similar_user_ids = users_sim_mat.loc[round(users_sim_mat.loc[user_id].sort_values(ascending=False), 5) >= 1][1:].index.values
    
users = []

for i in similar_user_ids:
    try:
        u_features = literal_eval(user_factors_df.loc[i, 'features'])
        user = np.array(u_features)
        users.append(user)
    except:
        continue

np.mean(users, axis=0)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
len(users)

In [ ]:
users_sim_mat.

In [ ]:
users_sim_mat[(round(users_sim_mat, 5) >= 1).sum() ==0]

In [ ]:
# Setup a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# Convert a Pandas DF to a Spark DF
spark_df = spark.createDataFrame(training) 

spark_df = spark_df.drop('timestamp')

spark_df.printSchema()

In [ ]:
train, validation = spark_df.randomSplit([0.8, 0.2], seed=111)

In [ ]:
train.describe().show()

In [ ]:
num_users = train.select('user').distinct().count()
num_movies = train.select('movie').distinct().count()
density = train.count() / (num_users*num_movies)
density

In [ ]:
als_model = ALS(userCol='user',
                itemCol='movie',
                ratingCol='rating',
                nonnegative=True,
                regParam=0.1,
                rank=10
)

recommender = als_model.fit(train)

predictions = recommender.transform(train)

predictions.describe().show()

In [ ]:
# recommender.itemFactors.show()

In [ ]:
# recommender.userFactors.show()

In [ ]:
# user1 = recommender.recommendForUserSubset(predictions, 10).where('user==700').select('recommendations').take(1)

# user1

In [ ]:
def predicted_rating(user_id, movie_id):
    try:
        user = recommender.userFactors.where(f'id == {user_id}').collect()[0]['features']
        item = recommender.itemFactors.where(f'id == {movie_id}').collect()[0]['features']
    except:
        user = 1
        item = -1
        
    return np.dot(np.array(user), np.array(item))

In [ ]:
predicted_rating(4615, 34)